Code to download the USGS earthquake catalog, and match it's earthquakes with CMT focal mechanisms.

In [ ]:
import numpy as np
from libcomcat.search import search
from datetime import datetime
import datetime as dt
import obspy

number_to_month = {'01':'jan', '02':'feb','03':'mar','04':'apr','05':'may','06':'jun','07':'jul','08':'aug','09':'sep','10':'oct','11':'nov','12':'dec'} # Used to convert a date to month name, used to navigate cmt data below
filename = 'usgs_full_catalog_cmt.csv' # Filename to store earthquake catalog
times_filename = 'usgs_full_catalog_times_cmt.txt' # Filename to store earthquake catalog time information

In [ ]:
# Download the USGS catalog with libcomcat search. The end time listed here was the time that we conducted the final download for the paper. 
X = search(starttime = datetime(1971,1,1), endtime= datetime(2021,1,18,19,4,21),minlatitude=31.354, maxlatitude=49.021, minlongitude=-124.915, maxlongitude=-104.766, minmagnitude = 1.0,eventtype = 'earthquake')

In [ ]:
# initiate arrays to save the required catalog information
longitude = np.zeros((len(X)))
latitude = np.zeros((len(X)))
magnitude = np.zeros((len(X)))
depth = np.zeros((len(X)))
times = np.zeros((len(X)))
strike1 = np.ones((len(X))) * np.nan
dip1 = np.ones((len(X))) * np.nan
strike2 = np.ones((len(X))) * np.nan
dip2 = np.ones((len(X))) * np.nan
rake1 = np.ones((len(X))) * np.nan
rake2 = np.ones((len(X))) * np.nan
times_1 = [] # List to save time information
list1 = [] # List used to hold the CMT catalog information for a particular earthquake
option = ''
# Download the cmt catalogs, see https://www.globalcmt.org/CMTfiles.html
cat76 = obspy.read_events('https://www.ldeo.columbia.edu/~gcmt/projects/CMT/catalog/jan76_dec17.ndk')
quickcat = obspy.read_events('https://www.ldeo.columbia.edu/~gcmt/projects/CMT/catalog/NEW_QUICK/qcmt.ndk')
for i in range(0,len(X)):
    dt = X[i].time # save the event time
    times_1.append(str(X[i].time))
    longitude[i] = X[i].longitude # extract event longitude
    latitude[i] = X[i].latitude # extract event latitude
    magnitude[i] = X[i].magnitude # extract event magnitude
    depth[i] = X[i].depth # extract event depth
    if magnitude[i] >= 5.0 and int(str(X[i].time)[0:4]) >= 1981: # If above M5, find focal mechanism.
        print('Year:',str(X[i].time)[0:4]) 
        if int(str(X[i].time)[0:4]) < 2006: # ALl CMTs before 2006 are in one data file, afterward CMT splits them by month. 
            cat = cat76
        else: # Use the monthly CMT catalog
            year = str(X[i].time)[0:4]
            month = str(X[i].time)[5:7]
            try: # Download the catalog for the month of interest
                cat = obspy.read_events('https://www.ldeo.columbia.edu/~gcmt/projects/CMT/catalog/NEW_MONTHLY/'+year+'/'+number_to_month[month]+year[2:4]+'.ndk')
            except: # If the month catalog does not exsist yet, the event is recent and might be in the quickcat catalog with the quick CMT solutions
                cat = quickcat
        for j in cat: # Search for the earthquake in the appropriate CMT catalog to get the CMT focal mechanism, we allow for slight devations in the event information since ComCat and CMT have slight discrepancies
            if (j.origins[0].longitude > longitude[i]-0.5) and (j.origins[0].longitude < longitude[i]+0.5):
                if str(j.origins[0].time)[0:10] == str(X[i].time)[0:10]:
                    if (j.origins[0].latitude > latitude[i]-0.5) and (j.origins[0].latitude < latitude[i]+0.5):
                        if (j.magnitudes[0].mag > magnitude[i]-0.5) and (j.magnitudes[0].mag < magnitude[i]+0.5):
                            list1.append(j) # If all of the event information matches, save the earthquake
                            
        if len(list1) == 0: # No Match between USGS catalog and CMT
            print()
            print('NEW EVENT')
            print(X[i].time,X[i].magnitude,X[i].latitude,X[i].longitude)
            print('No Event Found')
            option = 0 # Marks that a CMT solution cannot be matched to the earthquake
            Z = X[i].getDetailEvent()
            a = Z.toDict(get_tensors = 'preffered')
            for key in a.keys(): # In this case, we see if the USGS has a focal mechanism
                if 'np1_strike' in key:
                    strike1[i] = a[key]
                if 'np1_dip' in key:
                    dip1[i] = a[key]
                if 'np2_strike' in key:
                    strike2[i] = a[key]
                if 'np2_dip' in key:
                      dip2[i] = a[key]
                if 'np1_rake' in key:
                    rake1[i] = a[key]
                if 'np2_rake' in key:
                    rake2[i] = a[key]
            print('USGS Params', strike1[i],dip1[i],strike2[i],dip2[i],rake1[i],rake2[i])
                
        if len(list1) > 1: # Here there are multiple CMT events that may be a match
            #First, Use a more specific time criteria to differentiate between similar events
            match_key = False
            for w in range(len(list1)): # Check each event to see if it matches the more specific time criterion
                if int(str(list1[w].origins[0].time)[11:13]) == int(str(X[i].time)[11:13]) and int(str(list1[w].origins[0].time)[14:16]) == int(str(X[i].time)[14:16]):
                    match_key = True # If it does match, this key is turned on so we can skip the code for the case of no match
                    index = np.copy(w)
            if match_key == False: # If the more specific time criteria isn't met, the user is given the relative information and prompted to make the decision
                print()
                print("NEW EVENT")
                print('Length',len(list1))
                print('Multiple Events Found - Manual Differentiation Required')
                print('Original')
                print(X[i].magnitude)
                print(X[i].time)
                print(X[i].latitude)
                print(X[i].longitude)
                for w in range(len(list1)):
                    print()
                    print('Index '+str(w))
                    print(list1[w].magnitudes[0].mag)
                    print(list1[w].origins[0])
                if X[i].id == 'ci3031935': # For the current search settings, this is the only earthquake that still has multiple CMT matches by this stage, we provide the correct input so the user can replicate our work without providing any user input
                    index = 's'
                else:
                    print('Use the skip option when none of the possible CMT matches are correct')
                    index = input('Enter Correct Event Index, or S to skip:')
                if index.lower() == 's': # If there are no matches, try the USGS for a focal mechanism
                    option = 0 # Marks that a CMT solution cannot be matched to the earthquake
                    Z = X[i].getDetailEvent()
                    a = Z.toDict(get_tensors = 'preffered')
                    for key in a.keys(): # In this case, we see if the USGS has a focal mechanism
                        if 'np1_strike' in key:
                            strike1[i] = a[key]
                        if 'np1_dip' in key:
                            dip1[i] = a[key]
                        if 'np2_strike' in key:
                            strike2[i] = a[key]
                        if 'np2_dip' in key:
                              dip2[i] = a[key]
                        if 'np1_rake' in key:
                            rake1[i] = a[key]
                        if 'np2_rake' in key:
                            rake2[i] = a[key]
                else:
                    index = int(index)
            if option != 0:
                list1 = [list1[index]]
        if option != 0: # Extract the CMT focal mechanism if there was a match
            strike1[i] = list1[0].focal_mechanisms[0].nodal_planes.nodal_plane_1.strike
            dip1[i] = list1[0].focal_mechanisms[0].nodal_planes.nodal_plane_1.dip
            rake1[i] = list1[0].focal_mechanisms[0].nodal_planes.nodal_plane_1.rake
            strike2[i] = list1[0].focal_mechanisms[0].nodal_planes.nodal_plane_2.strike
            dip2[i] = list1[0].focal_mechanisms[0].nodal_planes.nodal_plane_2.dip
            rake2[i] = list1[0].focal_mechanisms[0].nodal_planes.nodal_plane_2.rake
        list1 = [] # reset list1 and option for the next earthquake
        option = ''             
# Initialize and populate a single array to hold all the data              
data_csv = np.ones(((len(X)),10))

data_csv[:,0] = latitude
data_csv[:,1] = longitude
data_csv[:,2] = depth
data_csv[:,3] = magnitude
data_csv[:,4] = strike1
data_csv[:,5] = dip1
data_csv[:,6] = rake1
data_csv[:,7] = strike2
data_csv[:,8] = dip2
data_csv[:,9] = rake2
# Save the data files
np.savetxt("usgs_full_catalog_cmt.csv", data_csv, delimiter=",",header = 'Longitude, Latitude, Depth (km), Magnitude, NP1 Strike, NP1 Dip, NP1 rake, NP2 Strike, NP2 Dip, NP2 Rake')
file = open('usgs_full_catalog_times_cmt.txt','w')
for i in times_1:
    file.write(i + '\n')
file.close()